# Check available LSMS data
Apparently not every country has not a lot of LSMS. In the following code we will check how many countries do have a LSMS and which ones have at least two, so we can perform the time travel. For checking this we are using the [search API](https://microdata.worldbank.org/api-documentation/catalog/index.html#operation/CatalogSearch) provided by the World Bank. We search for the keyword (the parameter `sk`) LSMS and specify the country by using the parameter `country`. For the country we use the ISO.

In [20]:
from tqdm import tqdm 
import pandas as pd
import requests

In [21]:
base_url: str = "https://microdata.worldbank.org/index.php/api/catalog/search?sk=lsms&country="

In [22]:
df: pd.DataFrame = pd.read_csv("../../data/countries_meta/countries_code.csv", index_col=False)

In [24]:
time_valid: pd.DataFrame = pd.DataFrame() # dataframe with country, iso, year and url for time travel
valid: pd.DataFrame = pd.DataFrame() # dataframe with country, iso, year and url for only one LSMS

for _, country in tqdm(df.iterrows(), total=len(df)):
    result: any = requests.get(base_url + country["iso"]).json()
    result = result["result"]

    if result["found"] == 0:
        continue
    
    count_valid: int = 0 # counter to count valid results
    years = [] # save years
    urls = [] # save urls
    for res in result["rows"]:

        if len(res["nation"].split(",")) > 1: # only if the country is alone in the `nation` tag, it is the desired data
            continue
        
        years.append(res["year_end"])
        urls.append(res["url"])
        count_valid += 1
    
    if count_valid > 0: # we just need to create a dataframe when, we found something
        tmp_df: pd.DataFrame = pd.DataFrame.from_dict({"name": country["name"], "iso": country["iso"], "year": years, "url": urls})
       
    if count_valid == 1:
        valid = valid.append(tmp_df)
    
    if count_valid > 1:
        time_valid = time_valid.append(tmp_df)

 14%|█▎        | 7/51 [00:05<00:44,  1.02s/it]/var/folders/v2/6cxx8khs0hs5nml1jx68qgk40000gn/T/ipykernel_10176/1422777032.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  valid = valid.append(tmp_df)
 18%|█▊        | 9/51 [00:07<00:34,  1.20it/s]/var/folders/v2/6cxx8khs0hs5nml1jx68qgk40000gn/T/ipykernel_10176/1422777032.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_valid = time_valid.append(tmp_df)
 35%|███▌      | 18/51 [00:12<00:20,  1.63it/s]/var/folders/v2/6cxx8khs0hs5nml1jx68qgk40000gn/T/ipykernel_10176/1422777032.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  valid = valid.append(tmp_df)
 39%|███▉      | 20/51 [00:13<00:18,  1.64it/s]/var/folders/v2/6cxx8khs0hs5nml1jx68qgk40000gn/T/ipykernel_1017

In [19]:
valid.to_csv("../../data/countries_meta/counties_lsms_valid.csv", index=False)
time_valid.to_csv("../../data/countries_meta/counties_lsms_time_valid.csv", index=False)